In [1]:
### THIS SCRIPT CLEANS THE COLLECTED SEP ###   

In [1]:
import pymongo 
import requests
import json

from tqdm import tqdm
from bson.objectid import ObjectId
from pprint import pprint

import lib_sepinpho as sep

##### INIT GLOBAL VARIABLES#####

#init Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#create database
db = client.visualizing_sep

In [32]:
##### Cleaning Sep_Entries ########
##### Completed 7.16.2020 ########

#1. Check for entries with InPhO apis.

sep_no_inpho = list(db.sep_entries.find( filter={'inpho_api': 'Error: No InPhO entry'},
                                         projection = ['title'],
                                         sort=[('title',1)]
                                        ))
pprint(sep_no_inpho)

#There were 15 SEP articles that originally didn't link into InPhO at all. I went into InPhO and found the appropriate API endpoint for those, and indicated the URL for those articles that do have InPhO entries. Not all of them had InPhO entries though. When we run this test now. We only see 4 articles left that don't link into InPhO

#Original Problem Articles
# [{'_id': ObjectId('5f0e28bc4386af32c64aa3d3'), 'title': 'Algebraic Propositional Logic'},
#  {'_id': ObjectId('5f0e229a4386af32c64aa0e4'), 'title': 'Basil [Cardinal] Bessarion'},
#  {'_id': ObjectId('5f0e230c4386af32c64aa119'), 'title': 'Catharine Macaulay'},
#  {'_id': ObjectId('5f0e237b4386af32c64aa152'), 'title': 'Communitarianism'},
#  {'_id': ObjectId('5f0e27524386af32c64aa31f'), 'title': 'Divine Illumination'},
#  {'_id': ObjectId('5f0e24ec4386af32c64aa1f5'), 'title': 'Dynamic Epistemic Logic'},
#  {'_id': ObjectId('5f0e25d74386af32c64aa26e'), 'title': 'Feminist Moral Psychology'},
#  {'_id': ObjectId('5f107e63b539c255926f6024'), 'title': 'Gaṅgeśa'},
#  {'_id': ObjectId('5f0e26ca4386af32c64aa2e2'), 'title': 'Hegel’s Dialectics'},
#  {'_id': ObjectId('5f0e27184386af32c64aa302'), 'title': 'Hume on Free Will'},
#  {'_id': ObjectId('5f0e27384386af32c64aa313'), 'title': 'Ibn Sina [Avicenna]'},
#  {'_id': ObjectId('5f0e27374386af32c64aa312'), 'title': 'Ibn Sina’s Natural Philosophy'},
#  {'_id': ObjectId('5f0e2c4c4386af32c64aa577'), 'title': 'Identity and Individuality in Quantum Theory'},
#  {'_id': ObjectId('5f0e279e4386af32c64aa347'), 'title': 'Intuitionism in Ethics'},
#  {'_id': ObjectId('5f0e28294386af32c64aa38d'), 'title': 'Kumārila'},
#  {'_id': ObjectId('5f0e23384386af32c64aa12f'), 'title': 'Legalism in Chinese Philosophy'},
#  {'_id': ObjectId('5f0e22de4386af32c64aa102'), 'title': 'Martin Buber'},
#  {'_id': ObjectId('5f0e2ab24386af32c64aa4ac'), 'title': 'Nietzsche’s Life and Works'},
#  {'_id': ObjectId('5f0e24384386af32c64aa1a5'), 'title': 'Religious Daoism'},
#  {'_id': ObjectId('5f0e2db54386af32c64aa62b'), 'title': 'Sophie de Grouchy'},
#  {'_id': ObjectId('5f0e2de04386af32c64aa642'), 'title': 'Spinoza’s Psychological Theory'},
#  {'_id': ObjectId('5f0e2e174386af32c64aa65b'), 'title': 'Supererogation'},
#  {'_id': ObjectId('5f0e2e5b4386af32c64aa682'), 'title': 'Touch'}]

#Ones that just didn't have a sibling article at InPhO
# [{'_id': ObjectId('5f109bb0789a3d802df9a834'), 'title': 'Hegel’s Dialectics'},
#  {'_id': ObjectId('5f109bdc789a3d802df9a864'), 'title': 'Ibn Sina’s Natural Philosophy'},
#  {'_id': ObjectId('5f109d6c789a3d802df9a9fe'), 'title': 'Nietzsche’s Life and Works'},
#  {'_id': ObjectId('5f109ee6789a3d802df9ab7d'), 'title': 'Sophie de Grouchy'}]


[{'_id': ObjectId('5f109bb0789a3d802df9a834'), 'title': 'Hegel’s Dialectics'},
 {'_id': ObjectId('5f109bdc789a3d802df9a864'),
  'title': 'Ibn Sina’s Natural Philosophy'},
 {'_id': ObjectId('5f109d6c789a3d802df9a9fe'),
  'title': 'Nietzsche’s Life and Works'},
 {'_id': ObjectId('5f109ee6789a3d802df9ab7d'), 'title': 'Sophie de Grouchy'}]


In [28]:
#Update the individual records identified above
title = 'Gaṅgeśa'
api_endpoint = 'thinker/3054'
collection_to_update = db.sep_entries

sep.update_sep_json(title,api_endpoint, collection_to_update)


acknowledged: http://inpho.cogs.indiana.edu/thinker/3054.json
 True
[{'_id': ObjectId('5f10a241b539c255926f702e'), 'inpho_api': 'http://inpho.cogs.indiana.edu/thinker/3054.json', 'inpho_json': {'wiki': 'Gangesha_Upadhyaya', 'influenced': [], 'death': [], 'teachers': [], 'url': '/thinker/3054', 'influenced_by': [], 'professions': [], 'death_strings': [], 'related_thinkers': [], 'sep_dir': '', 'students': [], 'birth_strings': [], 'birth': [], 'nationalities': [], 'related_ideas': [], 'label': 'Gangeśa', 'type': 'thinker', 'ID': 3054, 'aliases': []}, 'title': 'Gaṅgeśa'}]


In [31]:
#update Sophie de Grouchy to set her entry to a 'thinker' 
result =db.sep_entries.update_one({'title': 'Sophie de Grouchy'}, {'$set': {'inpho_JSON.type':'thinker'}})
# confirmation udpate
print (f'acknowledged', result.acknowledged)

acknowledged True


In [67]:
##### Cleaning Sep_Entries ########
##### Completed 7.16.2020 ########

#2. Check for duplicate InPhO JSON data.

collection_to_clean = db.sep_entries
result = list(collection_to_clean.aggregate([
                                {
                                    '$group' : { '_id' : "$inpho_json.ID",
                                    'count': { '$sum': 1 }
                                    }
                                },
                                {
                                    '$match': { 'count': {'$ne': 1 }}
                                }]))

pprint(result)

[{'_id': None, 'count': 4}]


In [66]:
# JSON.ID 3563 appears twice, so there's a duplicate entry. That is:
#Mencius (SEP) properly linked to Mencius (InPho) 
#Wang Yangming (SEP) improperly linked to Mencius (InPho)

title = 'Wang Yangming'
api_endpoint = 'thinker/4092'
collection_to_update = db.sep_entries

sep.update_sep_json(title,api_endpoint, collection_to_update)



acknowledged: http://inpho.cogs.indiana.edu/thinker/4092.json
 True
[{'_id': ObjectId('5f109f6c789a3d802df9ac08'), 'title': 'Wang Yangming', 'inpho_api': 'http://inpho.cogs.indiana.edu/thinker/4092.json', 'inpho_json': {'wiki': 'Wang_Yangming', 'influenced': [], 'death': [{'month': 0, 'day': 0, 'year': 1529}], 'teachers': [], 'url': '/thinker/4092', 'influenced_by': [4173], 'professions': [], 'death_strings': ['1529'], 'related_thinkers': [4173, 3563, 2844, 4203, 4195, 3060, 3177, 4198], 'sep_dir': 'wang-yangming', 'students': [], 'birth_strings': ['1472'], 'birth': [{'month': 0, 'day': 0, 'year': 1472}], 'nationalities': ['Chinese'], 'related_ideas': [6347, 148, 1870, 1343, 5338, 5510, 5602, 1544, 681], 'label': 'Wang Yangming', 'type': 'thinker', 'ID': 4092, 'aliases': []}}]
